Download all dependencies

In [9]:
%pip install stanza -U
%pip install accelerate -U
%pip install -U pip setuptools wheel textacy

Import Libraries

In [2]:
import Linguistic_Metrics as lm
import json
import timeit
import string
import nltk
import spacy
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import numpy as np

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\aryam\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aryam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Helper Functions

In [3]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

def replace_punctuation_with_whitespace(input_string):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    return input_string.translate(translator)

Initialize class instances 

In [4]:
%cd D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data
data = json.load(open(r'pairwise_evaluation_w_embeddings.json'))
data_nat = json.load(open(r'final_metrics_naturalness.json'))
df = pd.DataFrame(data_nat)
nat = lm.naturalness(df)
model = lm.LinguisticFeatures()

D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data


2024-03-04 16:53:22 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-03-04 16:53:27 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus                   |
| ner          | ontonotes-ww-multi_charlm |

2024-03-04 16:53:27 INFO: Using device: cpu
2024-03-04 16:53:27 INFO: Loading: tokenize
2024-03-04 16:53:27 INFO: Loading: mwt
2024-03-04 16:53:28 INFO: Loading: pos
2024-03-04 16:53:28 INFO: Loading: lemma
2024-03-04 16:53:28 INFO: Loading: constituency
2024-03-04 16:53:29 INFO: Loading: depparse
2024-03-04 16:53:30 INFO: Loading: sentiment
2024-03-04 16:53:31 INFO: Loading: ner
2024-03-04 16:53:32 INFO: Done loading processors!


Example for calculating naturalness for any text 

Problem: If we use very few words in a sentence and use more number of sentences, the score exceeds 1 sometimes

Potential Reason: The range for min max scaling is based on the dataset, for custom text the min max might change leading to score above 1 or less than 0.

In [4]:
score = nat.calculate_naturalness('Evans said officers negotiated with the suspect while additional officers responded to the scene.During those negotiations, the man opened fired, killing Elmstrand, Ruge and Finseth, and injuring Medlicott.')
score

0.7154226525399803

System Analysis

Generate all UPOS tags for the dataset

In [10]:
tags_writer,tags_llm,tags_article = model.spacy_tagger_dataset(data)

save_json(tags_writer,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/tags_writer.json')
save_json(tags_llm,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/tags_llm.json')
save_json(tags_article,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/tags_article.json')

KeyboardInterrupt: 

In [17]:
tags_writer = json.load(open(r'tags_writer.json'))
tags_llm= json.load(open(r'tags_llm.json'))
tags_article= json.load(open(r'tags_article.json'))

Calculate percentage of tags used 

In [18]:
tags_per_writer = model.count_tags_percentage(tags_writer)
tags_per_llm = model.count_tags_percentage(tags_llm)
tags_per_article = model.count_tags_percentage(tags_article)

print(tags_per_writer,tags_per_llm,tags_per_article)

save_json(tags_per_writer,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/tags_percentage_writer.json')
save_json(tags_per_llm,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/tags_percentage_llm.json')
save_json(tags_per_article,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/tags_percentage_article.json')

{'DET': 9.623379799461972, 'NOUN': 20.78136463683052, 'ADP': 11.65321594521888, 'PROPN': 9.280997798972853, 'VERB': 11.335289801907557, 'CCONJ': 2.3691611640988017, 'PRON': 4.01076057715823, 'PUNCT': 9.403277084861825, 'AUX': 5.5392516507703595, 'ADJ': 6.807899241868427, 'SCONJ': 1.7057960381511372, 'SPACE': 0.9384935191978478, 'NUM': 1.6966250917094645, 'ADV': 2.528124235754463, 'PART': 2.3080215211543167, 'X': 0.018341892883345562} {'DET': 10.157815631262524, 'NOUN': 20.052605210420843, 'ADP': 11.062750501002004, 'PROPN': 12.033441883767534, 'AUX': 5.573647294589178, 'VERB': 11.137900801603207, 'PART': 2.6928857715430863, 'PUNCT': 9.772670340681362, 'PRON': 3.547720440881764, 'ADJ': 5.92121743486974, 'ADV': 1.8881513026052104, 'CCONJ': 2.3327905811623246, 'NUM': 1.7566382765531061, 'SCONJ': 1.8474448897795592, 'SYM': 0.2035320641282565, 'X': 0.018787575150300603} {'PROPN': 9.368373536250905, 'PART': 2.6149282492669403, 'NOUN': 17.220934599292164, 'AUX': 4.8357300822013425, 'VERB': 10

Get all writer and LLM summaries

In [19]:
writer_summs = []
llm_summs = []
article_summs = []
rem_list = ['full_embedding', 'writer_summary_embedding', 'llm_summary_embedding']
for key in rem_list:
    for datum in data:
        writer_summs.append(datum['writer_summary'])
        llm_summs.append(datum['text-davinci-002_summary'])
        article_summs.append(datum['article_text'])
        del datum[key]

Calculate average dependency Tree heights for the whole dataset.(run the cell above before running this)

In [12]:
tree_ht_article = model.calculate_average_tree_height(article_summs)
tree_ht_writer = model.calculate_average_tree_height(writer_summs)
tree_ht_llm = model.calculate_average_tree_height(llm_summs)
print(tree_ht_writer,tree_ht_llm,tree_ht_article)

3.8430239667400343 4.02686623246493 4.0256846002404965


Count dependency arcs based metrics

In [20]:
arcs_writer = model.count_dependency_arcs(writer_summs)
arcs_llm = model.count_dependency_arcs(llm_summs)
arcs_article = model.count_dependency_arcs(article_summs)

print(arcs_writer,arcs_llm,arcs_article)

# save_json(arcs_writer,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/arcs1_writer.json')
# save_json(arcs_llm,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/arcs_llm1.json')
# save_json(arcs_article,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/arcs_article1.json')

{'number of sentences': 4788, 'average left arc length': 1.9764058198977585, 'average right arc length': 3.7244923697818137, 'average total arc length': 2.8673029952436044, 'right arc percentage': 50.964134207481685, 'left arc percentage': 49.035865792518315, 'standard deviation left arcs': 2.1279911149463357, 'standard deviation right arcs': 4.595775991690084, 'standard deviation total arcs': 3.7078829718281128} {'number of sentences': 4146, 'average left arc length': 1.8653553215444731, 'average right arc length': 4.3774183112261404, 'average total arc length': 3.1360214701839366, 'right arc percentage': 50.58257511291484, 'left arc percentage': 49.41742488708516, 'standard deviation left arcs': 1.7973213647715085, 'standard deviation right arcs': 5.41783435367229, 'standard deviation total arcs': 4.2451420578712495} {'number of sentences': 67140, 'average left arc length': 2.1942593671623642, 'average right arc length': 4.147497985255052, 'average total arc length': 3.23120575523608

Calculate average subtree analysis for the whole dataset.

In [22]:
word_avg_llm = model.calculate_subtree_features(llm_summs)
print(word_avg_llm)
word_avg_article = model.calculate_subtree_features(article_summs)
print(word_avg_article)
word_avg_writer = model.calculate_subtree_features(writer_summs)
print(word_avg_writer)

ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'list'>

Calculate average sentence length for all summaries in the dataset

In [21]:
average_length_writer,average_length_llm,average_length_article = model.avg_sentence_length_dataset(data)
print(average_length_writer,average_length_llm,average_length_article)

18.154671717171716 20.263386396526773 19.716268486916952


Calculate average word lengths for all summaries in the dataset

In [22]:
word_avg_llm = model.average_word_length_system(llm_summs)
print(word_avg_llm)
word_avg_article = model.average_word_length_system(article_summs)
print(word_avg_article)
word_avg_writer = model.average_word_length_system(writer_summs)
print(word_avg_writer)

4.858397518558533
4.599166289651323
4.809794325998426


Calculate Gender Bias in the dataset

In [ ]:
writer_ratio, llm_ratio, article_ratio = model.gender_ratio_dataset(data)
print(writer_ratio, llm_ratio, article_ratio)

Generate all analysis metrics for single data points

Calculating all the linguistic features for each data point in the dataset and creating a new document for analysis.

In [ ]:
for i,datum in enumerate(data):
    print('{}/{}'.format(i, len(data)))
    avg_len_writer = model.average_word_length_system([datum['writer_summary']])
    avg_len_llm = model.average_word_length_system([datum['text-davinci-002_summary']])
    avg_len_article = model.average_word_length_system([datum['article_text']])
    tree_ht_article = model.calculate_average_tree_height([datum['article_text']])
    tree_ht_llm = model.calculate_average_tree_height([datum['text-davinci-002_summary']])
    tree_ht_writer = model.calculate_average_tree_height([datum['writer_summary']])
    tags_writer = model.spacy_tagger(datum['writer_summary'])
    tags_llm = model.spacy_tagger(datum['text-davinci-002_summary'])
    tags_article = model.spacy_tagger(datum['article_text'])
    tags_per_writer = model.count_tags_percentage([tags_writer])
    tags_per_llm = model.count_tags_percentage([tags_llm])
    tags_per_article = model.count_tags_percentage([tags_article])
    sen_len_writer = model.avg_sentence_length(datum['writer_summary'])
    sen_len_llm = model.avg_sentence_length(datum['text-davinci-002_summary'])
    sen_len_article = model.avg_sentence_length(datum['article_text'])
    arcs_writer = model.count_dependency_arcs([datum['writer_summary']])
    arcs_llm = model.count_dependency_arcs([datum['text-davinci-002_summary']])
    arcs_article = model.count_dependency_arcs([datum['article_text']])
    subtree_writer = model.calculate_subtree_features(datum['writer_summary'])
    subtree_llm = model.calculate_subtree_features(datum['text-davinci-002_summary'])
    subtree_article = model.calculate_subtree_features(datum['article_text'])
    gender_writer = model.gender_ratio_single(datum['writer_summary'])
    gender_llm = model.gender_ratio_single(datum['text-davinci-002_summary'])
    gender_article = model.gender_ratio_single(datum['article_text'])

    datum['average word lengths'] = {"writer":avg_len_writer,"LLM":avg_len_llm,"article":avg_len_article}
    datum['Average Dependency tree heights'] = {"writer":tree_ht_writer,"LLM":tree_ht_llm,"article":tree_ht_article}
    datum['Percentage of UPOS tags used'] = {"writer":tags_per_writer,"LLM":tags_per_llm,"article":tags_per_article}
    datum['average sentence lengths'] = {"writer":sen_len_writer,"LLM":sen_len_llm,"article":sen_len_article}
    datum['Arc analysis'] = {"writer":arcs_writer,"LLM":arcs_llm,"article":arcs_article}
    datum['Gender Ratios'] = {"writer":gender_writer,"LLM":gender_llm,"article":gender_article}
    datum['Left and right subtrees analysis'] = {"writer":model.calculate_subtree_features(datum['writer_summary']),"LLM":model.calculate_subtree_features(datum['text-davinci-002_summary']),"article":model.calculate_subtree_features(datum['article_text'])}


save_json(data,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/data_analysis.json')

0/599
1/599
2/599
3/599
4/599
5/599
6/599
7/599
8/599
9/599
10/599
11/599
12/599
13/599
14/599
15/599
16/599
17/599
18/599
19/599
20/599
21/599
22/599
23/599
24/599
25/599
26/599
27/599
28/599
29/599
30/599
31/599
32/599
33/599
34/599
35/599
36/599
37/599
38/599
39/599
40/599
41/599
42/599
43/599
44/599
45/599
46/599
47/599
48/599
49/599
50/599
51/599
52/599
53/599
54/599
55/599
56/599
57/599
58/599
59/599
60/599
61/599
62/599
63/599
64/599
65/599
66/599
67/599
68/599
69/599
70/599
71/599
72/599
73/599
74/599
75/599
76/599
77/599
78/599
79/599
80/599
81/599
82/599
83/599
84/599
85/599
86/599
87/599
88/599
89/599
90/599
91/599
92/599
93/599
94/599
95/599
96/599
97/599
98/599
99/599
100/599
101/599
102/599
103/599
104/599
105/599
106/599
107/599
108/599
109/599
110/599
111/599
112/599
113/599
114/599
115/599
116/599
117/599
118/599
119/599
120/599
121/599
122/599
123/599
124/599
125/599
126/599
127/599
128/599
129/599
130/599
131/599
132/599
133/599
134/599
135/599
136/599
137/599
138/59

In [7]:
analysis_data = json.load(open(r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/data_analysis.json'))

Create Delta values document for calculating naturalness(Not used in actual calculation, but used in analysis of metrics to be selected)

In [ ]:
list_articles = []
for i,datum in enumerate(analysis_data):
    print('{}/{}'.format(i, len(data)))
    natural_dict = {}
    natural_dict['article_id'] = datum['article_id']
    natural_dict['writer_summary'] = datum['writer_summary']
    natural_dict['text-davinci-002_summary'] = datum['text-davinci-002_summary']
    natural_dict['overall_writer_better'] = datum['overall_writer_better']
    natural_dict['informative_writer_better'] = datum['informative_writer_better']
    natural_dict['delta average word lengths'] = datum['average word lengths']['writer'] - datum['average word lengths']['LLM']
    natural_dict['delta Average Dependency tree heights'] = datum['Average Dependency tree heights']['writer'] - datum['Average Dependency tree heights']['LLM']
    natural_dict['delta_average_sentence_lengths'] = datum['average sentence lengths']['writer'] - datum['average sentence lengths']['LLM']
    natural_dict['delta_average_left_arc_length'] = datum['Arc analysis']['writer']['average left arc length'] - datum['Arc analysis']['LLM']['average left arc length']
    natural_dict['delta_average_right_arc_length'] = datum['Arc analysis']['writer']['average right arc length'] - datum['Arc analysis']['LLM']['average right arc length']
    natural_dict['delta_average_right_arc_length'] = datum['Arc analysis']['writer']['average total arc length'] - datum['Arc analysis']['LLM']['average total arc length']
    natural_dict['delta_avg_left_subtree_height'] = datum['Left and right subtrees analysis']['writer']['avg_left_subtree_height'] - datum['Left and right subtrees analysis']['LLM']['avg_left_subtree_height']
    natural_dict['delta_avg_right_subtree_height'] = datum['Left and right subtrees analysis']['writer']['avg_right_subtree_height'] - datum['Left and right subtrees analysis']['LLM']['avg_right_subtree_height']
    natural_dict['delta_num_left_subtrees'] = datum['Left and right subtrees analysis']['writer']['num_left_subtrees'] - datum['Left and right subtrees analysis']['LLM']['num_left_subtrees']
    natural_dict['delta_num_right_subtrees'] = datum['Left and right subtrees analysis']['writer']['num_right_subtrees'] - datum['Left and right subtrees analysis']['LLM']['num_right_subtrees']
    natural_dict['delta_no_sentences'] = datum['Arc analysis']['writer']['number of sentences'] - datum['Arc analysis']['LLM']['number of sentences']
    natural_dict['no_sentences_writer'] = datum['Arc analysis']['writer']['number of sentences']
    natural_dict['no_sentences_llm'] = datum['Arc analysis']['LLM']['number of sentences']
    list_articles.append(natural_dict)
save_json(list_articles,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/delta_metrics_natural.json')

0/599
1/599
2/599
3/599
4/599
5/599
6/599
7/599
8/599
9/599
10/599
11/599
12/599
13/599
14/599
15/599
16/599
17/599
18/599
19/599
20/599
21/599
22/599
23/599
24/599
25/599
26/599
27/599
28/599
29/599
30/599
31/599
32/599
33/599
34/599
35/599
36/599
37/599
38/599
39/599
40/599
41/599
42/599
43/599
44/599
45/599
46/599
47/599
48/599
49/599
50/599
51/599
52/599
53/599
54/599
55/599
56/599
57/599
58/599
59/599
60/599
61/599
62/599
63/599
64/599
65/599
66/599
67/599
68/599
69/599
70/599
71/599
72/599
73/599
74/599
75/599
76/599
77/599
78/599
79/599
80/599
81/599
82/599
83/599
84/599
85/599
86/599
87/599
88/599
89/599
90/599
91/599
92/599
93/599
94/599
95/599
96/599
97/599
98/599
99/599
100/599
101/599
102/599
103/599
104/599
105/599
106/599
107/599
108/599
109/599
110/599
111/599
112/599
113/599
114/599
115/599
116/599
117/599
118/599
119/599
120/599
121/599
122/599
123/599
124/599
125/599
126/599
127/599
128/599
129/599
130/599
131/599
132/599
133/599
134/599
135/599
136/599
137/599
138/59

Create document for the final metrics used for calculating naturalness

In [8]:
list_articles = []
for i,datum in enumerate(analysis_data):
    print('{}/{}'.format(i, len(data)))
    natural_dict = {}
    natural_dict['article_id'] = datum['article_id']
    natural_dict['writer_summary'] = datum['writer_summary']
    natural_dict['text-davinci-002_summary'] = datum['text-davinci-002_summary']
    natural_dict['overall_writer_better'] = datum['overall_writer_better']
    natural_dict['informative_writer_better'] = datum['informative_writer_better']
    natural_dict['writer average word lengths'] = datum['average word lengths']['writer'] 
    natural_dict['LLM average word lengths'] = datum['average word lengths']['LLM']
    natural_dict['Writer Average Dependency tree heights'] = datum['Average Dependency tree heights']['writer']
    natural_dict['LLM Average Dependency tree heights'] = datum['Average Dependency tree heights']['LLM']
    natural_dict['writer_average_sentence_lengths'] = datum['average sentence lengths']['writer']
    natural_dict['LLM_average_sentence_lengths'] = datum['average sentence lengths']['LLM']
    natural_dict['writer_average_left_arc_length'] = datum['Arc analysis']['writer']['average left arc length']
    natural_dict['LLM_average_left_arc_length'] = datum['Arc analysis']['LLM']['average left arc length']
    natural_dict['writer_average_right_arc_length'] = datum['Arc analysis']['writer']['average right arc length']
    natural_dict['LLM_average_right_arc_length'] = datum['Arc analysis']['LLM']['average right arc length']
    natural_dict['writer_average_total_arc_length'] = datum['Arc analysis']['writer']['average total arc length']
    natural_dict['LLM_average_total_arc_length'] = datum['Arc analysis']['LLM']['average total arc length']
    natural_dict['writer_avg_left_subtree_height'] = datum['Left and right subtrees analysis']['writer']['avg_left_subtree_height'] 
    natural_dict['LLM_avg_left_subtree_height'] = datum['Left and right subtrees analysis']['LLM']['avg_left_subtree_height']
    natural_dict['writer_avg_right_subtree_height'] = datum['Left and right subtrees analysis']['writer']['avg_right_subtree_height']
    natural_dict['LLM_avg_right_subtree_height'] = datum['Left and right subtrees analysis']['LLM']['avg_right_subtree_height']
    natural_dict['writer_num_left_subtrees'] = datum['Left and right subtrees analysis']['writer']['num_left_subtrees']
    natural_dict['LLM_num_left_subtrees'] = datum['Left and right subtrees analysis']['LLM']['num_left_subtrees']
    natural_dict['writer_num_right_subtrees'] = datum['Left and right subtrees analysis']['writer']['num_right_subtrees']
    natural_dict['LLM_num_right_subtrees'] = datum['Left and right subtrees analysis']['LLM']['num_right_subtrees']
    # natural_dict['delta_no_sentences'] = datum['Arc analysis']['writer']['number of sentences'] - datum['Arc analysis']['LLM']['number of sentences']
    # natural_dict['no_sentences_writer'] = datum['Arc analysis']['writer']['number of sentences']
    # natural_dict['no_sentences_llm'] = datum['Arc analysis']['LLM']['number of sentences']
    list_articles.append(natural_dict)

0/599
1/599
2/599
3/599
4/599
5/599
6/599
7/599
8/599
9/599
10/599
11/599
12/599
13/599
14/599
15/599
16/599
17/599
18/599
19/599
20/599
21/599
22/599
23/599
24/599
25/599
26/599
27/599
28/599
29/599
30/599
31/599
32/599
33/599
34/599
35/599
36/599
37/599
38/599
39/599
40/599
41/599
42/599
43/599
44/599
45/599
46/599
47/599
48/599
49/599
50/599
51/599
52/599
53/599
54/599
55/599
56/599
57/599
58/599
59/599
60/599
61/599
62/599
63/599
64/599
65/599
66/599
67/599
68/599
69/599
70/599
71/599
72/599
73/599
74/599
75/599
76/599
77/599
78/599
79/599
80/599
81/599
82/599
83/599
84/599
85/599
86/599
87/599
88/599
89/599
90/599
91/599
92/599
93/599
94/599
95/599
96/599
97/599
98/599
99/599
100/599
101/599
102/599
103/599
104/599
105/599
106/599
107/599
108/599
109/599
110/599
111/599
112/599
113/599
114/599
115/599
116/599
117/599
118/599
119/599
120/599
121/599
122/599
123/599
124/599
125/599
126/599
127/599
128/599
129/599
130/599
131/599
132/599
133/599
134/599
135/599
136/599
137/599
138/59

Define range(range is min in both LLM and Writer summaries same for max to account for texts not in dataset) for selected features and min max scaling for selected features. These are the final values used for naturalness

In [ ]:
for dat in list_articles:
    range_avg_dep_tree_ht = (min(df['Writer Average Dependency tree heights'].min(),df['LLM Average Dependency tree heights'].min()),max(df['Writer Average Dependency tree heights'].max(),df['LLM Average Dependency tree heights'].max()))
    range_avg_sentence_length = (min(df['writer_average_sentence_lengths'].min(),df['LLM_average_sentence_lengths'].min()),max(df['writer_average_sentence_lengths'].max(),df['LLM_average_sentence_lengths'].max()))
    range_avg_left_subtree_ht = (min(df['writer_avg_left_subtree_height'].min(),df['LLM_avg_left_subtree_height'].min()),max(df['writer_avg_left_subtree_height'].max(),df['LLM_avg_left_subtree_height'].max()))
    range_avg_right_subtree_ht = (min(df['writer_avg_right_subtree_height'].min(),df['LLM_avg_right_subtree_height'].min()),max(df['writer_avg_right_subtree_height'].max(),df['LLM_avg_right_subtree_height'].max()))
    dat['Writer Average Dependency tree heights'] = (dat['Writer Average Dependency tree heights']-range_avg_dep_tree_ht[0])/(range_avg_dep_tree_ht[1]-range_avg_dep_tree_ht[0])
    dat['LLM Average Dependency tree heights'] = (dat['LLM Average Dependency tree heights']-range_avg_dep_tree_ht[0])/(range_avg_dep_tree_ht[1]-range_avg_dep_tree_ht[0])
    dat['writer_average_sentence_lengths'] = (dat['writer_average_sentence_lengths']-range_avg_sentence_length[0])/(range_avg_sentence_length[1]-range_avg_sentence_length[0])
    dat['LLM_average_sentence_lengths'] = (dat['LLM_average_sentence_lengths']-range_avg_sentence_length[0])/(range_avg_sentence_length[1]-range_avg_sentence_length[0])
    dat['writer_avg_left_subtree_height'] = (dat['writer_avg_left_subtree_height']-range_avg_left_subtree_ht[0])/(range_avg_left_subtree_ht[1]-range_avg_left_subtree_ht[0])
    dat['LLM_avg_left_subtree_height'] = (dat['LLM_avg_left_subtree_height']-range_avg_left_subtree_ht[0])/(range_avg_left_subtree_ht[1]-range_avg_left_subtree_ht[0])
    dat['writer_avg_right_subtree_height'] = (dat['writer_avg_right_subtree_height']-range_avg_right_subtree_ht[0])/(range_avg_right_subtree_ht[1]-range_avg_right_subtree_ht[0])
    dat['LLM_avg_right_subtree_height'] = (dat['LLM_avg_right_subtree_height']-range_avg_right_subtree_ht[0])/(range_avg_right_subtree_ht[1]-range_avg_right_subtree_ht[0])
save_json(list_articles,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/final_metrics_naturalness.json')

Dataframe for final features

In [30]:
natural_metrics_dict = json.load(open(r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/final_metrics_naturalness.json'))
df_features = pd.DataFrame(natural_metrics_dict)
data_natural =  json.load(open(r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/metrics_natural.json'))
df_delta = pd.DataFrame(natural_metrics_dict)

In [31]:
y = df_features['overall_writer_better']
y_delta = df_delta['overall_writer_better']

In [32]:
df_delta = df_delta.drop(columns = ['article_id','writer_summary','text-davinci-002_summary','informative_writer_better','overall_writer_better'])
df_features = df_features.drop(columns = ['article_id','writer_summary','text-davinci-002_summary','informative_writer_better','overall_writer_better'])

Load delta values document for feature importance measurement (not used directly for naturalness)

In [33]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(df_features))
X.columns = df_features.columns
X_delta = pd.DataFrame(scaler.fit_transform(df_delta))
X_delta.columns = df_delta.columns

In [34]:
X_delta

,writer average word lengths,LLM average word lengths,Writer Average Dependency tree heights,LLM Average Dependency tree heights,writer_average_sentence_lengths,LLM_average_sentence_lengths,writer_average_left_arc_length,LLM_average_left_arc_length,writer_average_right_arc_length,LLM_average_right_arc_length,writer_average_total_arc_length,LLM_average_total_arc_length,writer_avg_left_subtree_height,LLM_avg_left_subtree_height,writer_avg_right_subtree_height,LLM_avg_right_subtree_height,writer_num_left_subtrees,LLM_num_left_subtrees,writer_num_right_subtrees,LLM_num_right_subtrees
0,-1.380766,0.697487,-0.507613,-0.211258,-0.682516,0.071023,0.566101,-0.561978,-1.443369,0.759060,-0.987307,0.293710,-0.295790,-0.223755,-0.465830,-0.255318,-1.203295,0.259309,0.777728,-0.359648
1,-0.037666,-0.182140,-0.966904,0.057608,-1.390306,-0.389731,-1.219428,-1.450071,1.792654,0.010318,-0.558887,0.055597,-0.599273,0.624472,-1.503209,-0.189624,-1.862942,0.114685,-3.854393,1.437736
2,-1.304261,-1.063210,0.560380,-0.714782,-0.525229,0.531777,0.010091,-0.254106,-0.625570,-0.030974,-0.316909,0.788859,-0.288345,-0.533501,1.150088,-0.584834,-0.763531,-1.042305,1.218882,0.924197
3,0.410970,0.018400,0.209724,0.855243,1.322888,-0.005769,-0.504732,0.361638,0.901575,-0.587637,1.101243,-0.185154,0.307234,1.247038,0.108719,0.496269,-0.763531,0.837804,0.777728,1.822889
4,-1.191187,-0.799278,-0.907963,-0.597923,-0.210656,0.646966,2.519856,0.777265,-0.167049,-0.368864,1.286891,0.655319,-1.339915,-0.449869,-0.465830,-0.490978,1.435291,-0.753057,0.336574,1.566120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,-1.144454,1.962989,-1.021915,-0.250097,-0.289300,-1.541616,0.029159,0.757473,-0.505391,-1.593287,-0.425475,-1.336503,-0.759846,-0.652974,-1.136137,0.312370,0.335880,-1.620800,0.777728,-1.258340
595,1.046883,-1.298389,0.207373,-0.591018,0.968993,0.877343,-0.195838,-0.430033,-0.016303,2.033148,0.126586,1.422319,0.232786,0.063866,0.149758,-1.342721,-0.763531,0.403932,0.336574,-0.488033
596,-0.920947,-0.143246,1.597009,-0.452924,1.794747,0.301400,-0.291448,0.109742,0.548030,-0.606792,0.198783,-0.317620,2.125878,-0.153358,0.860052,-0.554582,0.555762,-0.463810,-0.104581,0.282275
597,0.531846,-0.037441,-1.035668,-0.548699,-1.154376,1.222908,-0.379384,0.361638,-1.277091,4.497343,-1.575195,4.071414,-0.997214,-0.238331,-0.922857,-1.003215,-0.543649,-1.186929,-0.986890,-1.643494


In [35]:
encoder = LabelEncoder()
y = [str(value) for value in y]
y = encoder.fit_transform(y)
y_delta = [str(value) for value in y_delta]
y_delta = encoder.fit_transform(y_delta)

Random forest classifier training to assign weights to the features

In [46]:
feature_writer = [
    'writer average word lengths',
    'LLM average word lengths',
    'Writer Average Dependency tree heights',
    'LLM Average Dependency tree heights',
    'writer_average_sentence_lengths', 
    'LLM_average_sentence_lengths',
    'writer_average_left_arc_length', 
    'LLM_average_left_arc_length',
    'writer_average_right_arc_length', 
    'LLM_average_right_arc_length',
    'writer_average_total_arc_length', 
    'LLM_average_total_arc_length',
    'writer_avg_left_subtree_height',
    'LLM_avg_left_subtree_height',
    'writer_avg_right_subtree_height', 
    'LLM_avg_right_subtree_height',
    'writer_num_left_subtrees', 
    'LLM_num_left_subtrees',
    'writer_num_right_subtrees', 
    'LLM_num_right_subtrees'
]

# encode target variable
# encoder = LabelEncoder()
# df['overall_writer_better'] = [str(value) for value in df['overall_writer_better']]
# df['informative_writer_better'] = [str(value) for value in df['informative_writer_better']]
# df['overall_writer_better'] = encoder.fit_transform(df['overall_writer_better'])
# df['informative_writer_better'] = encoder.fit_transform(df['informative_writer_better'])

X_train, X_test, y_train, y_test = train_test_split(X[feature_writer],y, test_size=0.2, random_state=42) # change to feature_LLM for LLM weights

# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

importances = clf.feature_importances_
for feature, importance in zip(feature_writer, importances):
    print(f"{feature}: {importance}")


writer average word lengths: 0.04713643615700032
LLM average word lengths: 0.044033650184514314
Writer Average Dependency tree heights: 0.0511629389559367
LLM Average Dependency tree heights: 0.03765093414910327
writer_average_sentence_lengths: 0.052409502527632434
LLM_average_sentence_lengths: 0.04729637666416571
writer_average_left_arc_length: 0.05447772130382169
LLM_average_left_arc_length: 0.03580421957067919
writer_average_right_arc_length: 0.053002465694039096
LLM_average_right_arc_length: 0.03295351637958432
writer_average_total_arc_length: 0.05955238865311208
LLM_average_total_arc_length: 0.03285219951362089
writer_avg_left_subtree_height: 0.04554170258220313
LLM_avg_left_subtree_height: 0.037001635495882305
writer_avg_right_subtree_height: 0.04331582809076042
LLM_avg_right_subtree_height: 0.04706327086036865
writer_num_left_subtrees: 0.04548456660534071
LLM_num_left_subtrees: 0.0777367516484307
writer_num_right_subtrees: 0.05161128960746004
LLM_num_right_subtrees: 0.1039126053

Calculating naturalness for all writer and LLM summaries in the dataset(Same weights are taken for both to prevent bias in one category)

In [37]:
weights= {
    "LLM Average Dependency tree heights": 0.24572631036047535,
    "LLM_average_sentence_lengths": 0.2522139996530825,
    "LLM_avg_left_subtree_height": 0.2106586374520842,
    "LLM_avg_right_subtree_height": 0.2914010525343579
}

In [38]:
def calculate_naturalness_score(features, weights):
    inverses = 1.0-features
    naturalness_score = np.mean(np.dot(inverses, weights))
    return naturalness_score

df_features['naturalness_score_llm'] = df_features.apply(lambda row: calculate_naturalness_score(row[weights_llm.keys()], np.array(list(weights_llm.values()))), axis=1)
df_features['naturalness_score_writer'] = df_features.apply(lambda row: calculate_naturalness_score(row[weights_writer.keys()], np.array(list(weights_writer.values()))), axis=1)

In [39]:
csv = df_features.to_csv('df_features.csv', index = True) 

In [47]:
import pandas as pd
calculate_average_tree_height
from sklearn.preprocessing import StandardScaler

feature_cols = [
    'writer average word lengths',
    'LLM average word lengths',
    'Writer Average Dependency tree heights',
    'LLM Average Dependency tree heights',
    'writer_average_sentence_lengths', 
    'LLM_average_sentence_lengths',
    'writer_average_left_arc_length', 
    'LLM_average_left_arc_length',
    'writer_average_right_arc_length', 
    'LLM_average_right_arc_length',
    'writer_average_total_arc_length', 
    'LLM_average_total_arc_length',
    'writer_avg_left_subtree_height',
    'LLM_avg_left_subtree_height',
    'writer_avg_right_subtree_height', 
    'LLM_avg_right_subtree_height',
    'writer_num_left_subtrees', 
    'LLM_num_left_subtrees',
    'writer_num_right_subtrees', 
    'LLM_num_right_subtrees'
]
x = X_delta.loc[:, feature_cols].values
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(x)
importance = pca.components_[0]
feature_importance = pd.DataFrame(list(zip(feature_cols, importance)), columns=['Feature', 'Importance'])
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
print(feature_importance)

                                   Feature  Importance
2   Writer Average Dependency tree heights    0.425128
12          writer_avg_left_subtree_height    0.404898
14         writer_avg_right_subtree_height    0.395871
4          writer_average_sentence_lengths    0.348031
3      LLM Average Dependency tree heights    0.257133
15            LLM_avg_right_subtree_height    0.249657
13             LLM_avg_left_subtree_height    0.226743
1                 LLM average word lengths    0.192375
18               writer_num_right_subtrees    0.150841
8          writer_average_right_arc_length    0.141357
10         writer_average_total_arc_length    0.111973
19                  LLM_num_right_subtrees    0.044172
5             LLM_average_sentence_lengths    0.003687
16                writer_num_left_subtrees   -0.000502
17                   LLM_num_left_subtrees   -0.017979
0              writer average word lengths   -0.030280
6           writer_average_left_arc_length   -0.089290
7         

Experiment to use delta values for assigning weights(Problem: both LLM and writer summaries will have the same naturalness score)

In [44]:
from sklearn.metrics import confusion_matrix, accuracy_score

feature_cols = [
    'writer average word lengths',
    'LLM average word lengths',
    'Writer Average Dependency tree heights',
    'LLM Average Dependency tree heights',
    'writer_average_sentence_lengths', 
    'LLM_average_sentence_lengths',
    'writer_average_left_arc_length', 
    'LLM_average_left_arc_length',
    'writer_average_right_arc_length', 
    'LLM_average_right_arc_length',
    'writer_average_total_arc_length', 
    'LLM_average_total_arc_length',
    'writer_avg_left_subtree_height',
    'LLM_avg_left_subtree_height',
    'writer_avg_right_subtree_height', 
    'LLM_avg_right_subtree_height',
    'writer_num_left_subtrees', 
    'LLM_num_left_subtrees',
    'writer_num_right_subtrees', 
    'LLM_num_right_subtrees'
]
# df['overall_writer_better'] = [str(value) for value in df['overall_writer_better']]
# df['informative_writer_better'] = [str(value) for value in df['informative_writer_better']]
# encoder = LabelEncoder()
# df['overall_writer_better'] = encoder.fit_transform(df['overall_writer_better'])
# df['informative_writer_better'] = encoder.fit_transform(df['informative_writer_better'])


# Y_train = llm/human, x_train = linguistic features. 
# check metrics correlation and statistical differences
# contrast LLM and human summaries using graphs or statistical measures to select important features.

X_train, X_test, y_train, y_test = train_test_split(df_delta[feature_cols], y_delta, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_preds = clf.predict(X_test)
acc = accuracy_score(y_test, y_preds)
print (acc)
importances = clf.feature_importances_
for feature, importance in zip(feature_cols, importances):
    print(f"{feature}: {importance}")

0.425
writer average word lengths: 0.04713643615700032
LLM average word lengths: 0.044033650184514314
Writer Average Dependency tree heights: 0.0511629389559367
LLM Average Dependency tree heights: 0.03765093414910327
writer_average_sentence_lengths: 0.052409502527632434
LLM_average_sentence_lengths: 0.04729637666416571
writer_average_left_arc_length: 0.05447772130382169
LLM_average_left_arc_length: 0.03580421957067919
writer_average_right_arc_length: 0.053002465694039096
LLM_average_right_arc_length: 0.03295351637958432
writer_average_total_arc_length: 0.05955238865311208
LLM_average_total_arc_length: 0.03285219951362089
writer_avg_left_subtree_height: 0.04554170258220313
LLM_avg_left_subtree_height: 0.037001635495882305
writer_avg_right_subtree_height: 0.04331582809076042
LLM_avg_right_subtree_height: 0.04706327086036865
writer_num_left_subtrees: 0.04548456660534071
LLM_num_left_subtrees: 0.0777367516484307
writer_num_right_subtrees: 0.05161128960746004
LLM_num_right_subtrees: 0.1039

In [152]:
weights = {
    'delta Average Dependency tree heights': 0.15910590906248265,
    'delta_average_sentence_lengths': 0.1829254004041044,
    'delta_average_left_arc_length': 0.1412567431684343,
    'delta_average_right_arc_length': 0.18420709379634245,
    'delta_avg_left_subtree_height': 0.15108699260841774,
    'delta_avg_right_subtree_height': 0.18141786096021847,
    # 'delta_num_left_subtrees': 0.17379453913879672,
    # 'delta_num_right_subtrees': 0.16617592483149365
}

In [153]:
df = df_delta[weights.keys()]

In [154]:
df

,delta Average Dependency tree heights,delta_average_sentence_lengths,delta_average_left_arc_length,delta_average_right_arc_length,delta_avg_left_subtree_height,delta_avg_right_subtree_height
0,-0.337083,-5.000000,0.533333,-0.760000,-0.235185,-0.239130
1,-0.765152,-6.000000,0.065611,-0.504884,-0.927602,-1.006006
2,0.590533,-6.333333,0.176768,-0.729847,-0.045455,1.109848
3,-0.496496,3.833333,-0.272727,0.193662,-0.746334,-0.350000
4,-0.364286,-5.500000,0.912500,-0.097126,-0.737500,-0.078947
...,...,...,...,...,...,...
594,-0.627083,3.666667,-0.179894,0.135088,-0.261905,-1.091667
595,0.307692,-1.500000,0.149351,-0.840000,-0.084416,0.928571
596,1.072802,4.500000,-0.087662,-0.074074,1.201299,0.887363
597,-0.469086,-12.000000,-0.217391,-2.577778,-0.654731,-0.048951


In [155]:
df['naturalness'] = sum(df[column] * weight for column, weight in weights.items())/len(weights)

C:\Users\aryam\AppData\Local\Temp\ipykernel_25744\2960091211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naturalness'] = sum(df[column] * weight for column, weight in weights.items())/len(weights)


In [156]:
# cap for max values, seperate some data for testing 
# revert delta values.
# meta-data weights: mean sd, min, max, quantile ranges

# ranges = [range 1:desc]

# define classes for faithfulness
Q1 = np.percentile(df['naturalness'], 25)
Q2 = np.percentile(df['naturalness'], 50)
Q3 = np.percentile(df['naturalness'], 75)

# define ranges, add another range for all quartiles.
range1 = (df['naturalness'].min(), Q1)
range2 = (Q1,Q2)
range3 = (Q2,Q3)
range4 = (Q3, df['naturalness'].max())

print(f"Range 1: {range1}")
print(f"Range 2: {range2} (IQR)")
print(f"Range 3: {range3}")

Range 1: (-0.6128094968506038, -0.25532200174494046)
Range 2: (-0.25532200174494046, 0.09398677248950198) (IQR)
Range 3: (0.09398677248950198, 0.5512335540098786)


In [157]:
for ind in df.index:
    if df['naturalness'][ind] <= Q1:
        df['naturalness'][ind]= 'bad'
    elif Q1 < df['naturalness'][ind] <= Q3:
        df['naturalness'][ind]= 'avg'
    else:
        df['naturalness'][ind]= 'good'

C:\Users\aryam\AppData\Local\Temp\ipykernel_25744\297741487.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naturalness'][ind]= 'avg'
C:\Users\aryam\AppData\Local\Temp\ipykernel_25744\297741487.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naturalness'][ind]= 'bad'
C:\Users\aryam\AppData\Local\Temp\ipykernel_25744\297741487.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naturalness'][ind]= 'good'


In [158]:
df['naturalness'].value_counts()

avg     300
bad     151
good    148
Name: naturalness, dtype: int64

In [159]:
df['naturalness'][0]

'avg'

In [160]:
for ind, datum in enumerate(data_natural):
    datum['naturalness'] = df['naturalness'][ind]
data_natural

[{'article_id': '18cba9a8f2f64055a707452638182303',
  'writer_summary': 'The mayor of Baltimore fired the police chief and replaced him with his deputy. According to the mayor, crime in the city was unacceptable. Riots in the city after a man died in police custody and a surge in homicide rates were cited as reasons for the firing.   ',
  'text-davinci-002_summary': 'The mayor of Baltimore has sacked the city\'s police chief, saying his leadership has become a distraction from fighting a "crime surge". Mayor Stephanie Rawlings-Blake said she was replacing Police Commissioner Anthony Batts with his deputy, Kevin Davis, for an interim period.',
  'overall_writer_better': False,
  'informative_writer_better': 'Equally Good',
  'delta average word lengths': -0.8276515151515147,
  'delta Average Dependency tree heights': -0.3370827285921627,
  'delta_average_sentence_lengths': -5.0,
  'delta_average_left_arc_length': 0.5333333333333334,
  'delta_average_right_arc_length': -0.759999999999999

In [161]:
save_json(data_natural,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/naturalness_scores.json')